## 슬기로운 퀀트투자 텀프로젝트

- 팩터 계산 부분만 있습니다.

In [20]:
import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine, engine
import numpy as np
import warnings
from functools import reduce

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# 한개 종목 데이터 불러오기
# s1 = GetFactor.getFactor(stock_code='005930', start_quarter='2022Q1')
# # 한개 종목 팩터 불러오기
# s1.getPER()
# s1.getPBR()
# s1.getPSR()
# s1.getPCR()
# s1.getEVEBITDA()
# s1.getEVSales()
# s1.getGraham()

# 모든 종목 데이터 불러오기
# stocks = GetFactor.getFactor(stock_code='ALL', start_quarter='2022Q1')
# 모든 종목의 트레일링 데이터와 주가 데이터를 불러오는데 시간이 오래 소요되므로 DB에 저장된 데이터 불러옴
''' Params
start_quarter: 시작 기간
end_quarter: None, 종료 기간
stock_code: None, 'ALL': 모든 데이터 조회, 'LATER': 데이터 나중에 조회, 'LOAD': 모든 데이터 로딩, 종목코드
by: 'consolidated',
freq: 'q'
'''
stocks = getFactor(stock_code='LOAD', start_quarter='2022Q1')
per = stocks.getPER()
pbr = stocks.getPBR()
psr = stocks.getPSR()
pcr = stocks.getPCR()
evebitda = stocks.getEVEBITDA()
evsales = stocks.getEVSales()
graham = stocks.getGraham()
roa = stocks.getROA()
# ROE 4분기전 데이터 필요
#stocks.getROE()
# RIM 계수 입력을 통해 여러 계수에 따른 RIM 계산 가능
# 12분기 데이터 필요
#stocks.getRIM(0.1)
gpa = stocks.getGPA()
stable = stocks.getStabilityIdx()
# 성장율 지표 12분기간 데이터 필요
# stocks.getGrowthRate()
# 회전율 지표 4분기전 데이터 필요
# stocks.getTurnoverRatio()
moat = stocks.getMoatIdx()

Combo2_absolute = stocks.get2Combo('absolute')
Combo2_relative = stocks.get2Combo('relative')
Combo4 = stocks.get4Combo()
stocks.setMissingValue()

수정주가 데이터 로딩
start_date(2022-01-01) ~ end_date(2022-03-31)
수정주가 데이터 로딩 성공
저장된 트레일링 데이터 로딩


### 데이터 부족으로 인한 팩터 산출 불가 지표
  
---
1. ROE  
- ROE 4분기전 데이터 필요  
- stocks.getROE()  
2. RIM  
- RIM 계수 입력을 통해 여러 계수에 따른 RIM 계산 가능  
- 12분기 데이터 필요  
- stocks.getRIM(0.1)  
3. 성장율 지표  
- 성장율 지표 12분기간 데이터 필요  
- stocks.getGrowthRate()  
4. 회전율 지표  
- 회전율 지표 4분기전 데이터 필요  
- stocks.getTurnoverRatio()  

In [22]:
# 주가 데이터
stocks.priceData

,종목코드,종목명,시장,기간,시가,고가,저가,종가,거래량,거래대금,시가총액,상장주식수
0,000020,동화약품,KOSPI,2022Q1,14800.0000,15350.0000,11250.0000,13650.0000,214548.0000,2950613450.0000,381264565500.0000,27931470.0000
1,000040,KR모터스,KOSPI,2022Q1,867.0000,1045.0000,720.0000,889.0000,417614.0000,372592993.0000,85466120152.0000,96137368.0000
2,000050,경방,KOSPI,2022Q1,13750.0000,16000.0000,13400.0000,15950.0000,42169.0000,667150500.0000,437273556500.0000,27415270.0000
3,000060,메리츠화재,KOSPI,2022Q1,33300.0000,53500.0000,33300.0000,47450.0000,298202.0000,14046227500.0000,5723656250000.0000,120625000.0000
4,000070,삼양홀딩스,KOSPI,2022Q1,98300.0000,99000.0000,79900.0000,86100.0000,13667.0000,1170974500.0000,737383733100.0000,8564271.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
2326,950170,JTC,KOSDAQ,2022Q1,4005.0000,4660.0000,2700.0000,4585.0000,63761.0000,293978195.0000,160500295445.0000,35005517.0000
2327,950190,미투젠,KOSDAQ,2022Q1,23550.0000,23700.0000,16950.0000,20300.0000,45327.0000,914455800.0000,275671807600.0000,13579892.0000
2328,950200,소마젠,KOSDAQ,2022Q1,8890.0000,9600.0000,6520.0000,7290.0000,23924.0000,173970860.0000,139483601370.0000,19133553.0000
2329,950210,프레스티지바이오파마,KOSPI,2022Q1,20900.0000,22500.0000,12850.0000,20300.0000,511003.0000,10140785700.0000,1219951946500.0000,60096155.0000


In [23]:
# 트레일링 데이터
stocks.trailingData

,종목코드,종목명,시장,기간,컬럼,매출액,매출원가,매출총이익,판매비,관리비,...,재무활동으로인한현금흐름,재무활동으로인한현금유입액,재무활동으로인한현금유출액,기타재무활동으로인한현금흐름,영업투자재무활동기타현금흐름,연결범위변동으로인한현금의증가,환율변동효과,현금및현금성자산의증가,기초현금및현금성자산,기말현금및현금성자산
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,3064.8400,1483.1700,1581.6800,95.0500,232.8400,...,-77.4900,6.0000,33.6700,-49.8200,0.0000,0.0000,1.1700,160.7800,1991.0900,2151.8800
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,1399.6200,1218.5800,181.0200,17.1600,21.1500,...,43.2000,171.6000,128.4000,0.0000,-1.5100,0.0000,11.7300,-63.5900,964.5800,900.9900
2,000050,경방,KOSPI,2022Q1,consolidated_q,3975.7200,2535.1800,1440.5400,23.9800,670.9000,...,-88.5200,3864.3400,3920.3700,-32.4800,0.0000,0.0000,16.0400,30.9800,767.4800,798.4600
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,32289.5000,25241.5000,7047.9900,990.1700,809.9300,...,686.2300,9011.3200,7983.7900,-341.3000,0.0000,0.0000,5.3000,-60.9900,8384.7600,8323.7700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2177,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,948.1400,292.5400,655.5800,0.0000,19.9600,...,-129.6800,104.9400,228.5300,-6.1000,0.0000,0.0000,7.4100,253.7900,2061.5000,2352.7200
2178,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2179,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,0.0000,0.0000,0.0000,0.0000,0.0000,...,-40.2400,0.0000,40.2400,0.0000,0.0000,0.0000,-24.5900,-2268.2400,12888.9000,10718.6000


In [24]:
per

,종목코드,종목명,시장,기간,컬럼,EPS,PER
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,711.4914,19.1851
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,-134.0270,-6.6330
2,000050,경방,KOSPI,2022Q1,consolidated_q,1115.0720,14.3040
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,6168.1907,7.6927
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,27841.7159,3.0925
...,...,...,...,...,...,...,...
1809,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,9729.4471,1.8501
1810,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,698.8169,11.7770
1811,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,1977.9981,10.2629
1812,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,-1827.0387,-11.1109


In [25]:
pbr

,종목코드,종목명,시장,기간,컬럼,BPS,PBR
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,51003.0442,0.2676
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,2094.2533,0.4245
2,000050,경방,KOSPI,2022Q1,consolidated_q,110169.2597,0.1448
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,73541.8031,0.6452
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,1129448.1457,0.0762
...,...,...,...,...,...,...,...
2175,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,596.9259,13.4188
2176,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,34589.6713,0.5869
2177,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,1634.0405,4.4613
2178,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,37461.7977,0.5419


In [26]:
psr

,종목코드,종목명,시장,기간,컬럼,SPS,PSR
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,10972.7129,1.2440
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,1455.8543,0.6106
2,000050,경방,KOSPI,2022Q1,consolidated_q,14501.8451,1.0999
3,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,377025.6686,0.2284
4,000080,하이트진로,KOSPI,2022Q1,consolidated_q,32103.1524,1.1666
...,...,...,...,...,...,...,...
1807,900310,컬러레이,KOSDAQ,2022Q1,consolidated_q,1007.1111,1.4149
1808,900340,윙입푸드,KOSDAQ,2022Q1,consolidated_q,3306.2886,0.3796
1809,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,29889.9697,0.6022
1810,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,7973.9314,1.0321


In [27]:
pcr

,종목코드,종목명,시장,기간,컬럼,CFPS,PCR
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,1564.9731,8.7222
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,-89.2161,-9.9646
2,000050,경방,KOSPI,2022Q1,consolidated_q,1306.3158,12.2099
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,9186.1554,5.1654
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,23586.0122,3.6505
...,...,...,...,...,...,...,...
1808,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,19289.7104,0.9331
1809,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,-603.7891,-13.6306
1810,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,1753.4013,11.5775
1811,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,-581.0022,-34.9396


In [28]:
evebitda

,종목코드,종목명,시장,기간,컬럼,EV,EBITDA,EV/EBITDA
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,197906565500.0000,37138000000.0000,5.3290
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,30396120152.0000,199000000.0000,0.8185
2,000050,경방,KOSPI,2022Q1,consolidated_q,1182981556500.0000,99615000000.0000,31.8537
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,5723656250000.0000,1025690000000.0000,154.1186
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,1378534733100.0000,403924000000.0000,37.1193
...,...,...,...,...,...,...,...,...
2175,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,518607850500.0000,0.0000,13.9643
2176,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,41701807600.0000,34232000000.0000,1.1229
2177,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,139483601370.0000,0.0000,3.7558
2178,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,150964946499.9999,-27074000000.0000,4.0650


In [29]:
evsales

,종목코드,종목명,시장,기간,컬럼,EV,EV/Sales
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,197906565500.0000,0.6457
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,30396120152.0000,0.2172
2,000050,경방,KOSPI,2022Q1,consolidated_q,1182981556500.0000,2.9755
3,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,1378534733100.0000,0.4269
4,000080,하이트진로,KOSPI,2022Q1,consolidated_q,2678024731950.0000,1.1894
...,...,...,...,...,...,...,...
1807,900310,컬러레이,KOSDAQ,2022Q1,consolidated_q,-393791000000.0000,-7.2409
1808,900340,윙입푸드,KOSDAQ,2022Q1,consolidated_q,-249156347860.0000,-1.5708
1809,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,-341930514000.0000,-0.3150
1810,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,209658537180.0000,1.3234


In [30]:
graham

,종목코드,종목명,시장,기간,컬럼,NCAV,Graham
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,516129000000.0001,-55767848249.9999
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,-206722000000.0000,-334921180228.0000
2,000050,경방,KOSPI,2022Q1,consolidated_q,-1584064000000.0000,-2239974334750.0000
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,-99719152000000.0000,-108304636375000.0000
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,-1174469999999.9998,-2280545599650.0000
...,...,...,...,...,...,...,...
2175,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,20697000000.0000,-757214775750.0000
2176,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,392260000000.0001,-21247711399.9999
2177,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,12789000000.0000,-196436402055.0000
2178,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,1444532000000.0000,-385395919750.0000


In [31]:
roa

,종목코드,종목명,시장,기간,컬럼,ROA
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,0.0959
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,-0.1381
2,000050,경방,KOSPI,2022Q1,consolidated_q,0.3904
3,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,0.2854
4,000080,하이트진로,KOSPI,2022Q1,consolidated_q,0.0407
...,...,...,...,...,...,...
1809,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,0.4981
1810,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,0.3555
1811,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,0.1217
1812,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,-0.0930


In [32]:
gpa

,종목코드,종목명,시장,기간,컬럼,GP/A
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,0.0877
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,0.0262
2,000050,경방,KOSPI,2022Q1,consolidated_q,0.0282
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,0.0000
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,0.0400
...,...,...,...,...,...,...
2175,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,0.0000
2176,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,0.1246
2177,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,0.0000
2178,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,0.0000


In [33]:
stable

,종목코드,종목명,시장,기간,컬럼,부채비율,차입금비율
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,26.5437,2.2343
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,243.1920,17.3983
2,000050,경방,KOSPI,2022Q1,consolidated_q,69.3877,27.3333
3,000060,메리츠화재,KOSPI,2022Q1,consolidated_q,1124.3234,0.0000
4,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,82.2613,15.2336
...,...,...,...,...,...,...,...
2175,950160,코오롱티슈진,KOSDAQ,2022Q1,consolidated_q,58.8698,0.0000
2176,950190,미투젠,KOSDAQ,2022Q1,consolidated_q,11.9811,0.2772
2177,950200,소마젠,KOSDAQ,2022Q1,consolidated_q,45.0824,0.0000
2178,950210,프레스티지바이오파마,KOSPI,2022Q1,consolidated_q,9.6818,0.1276


In [34]:
moat

,종목코드,종목명,시장,기간,컬럼,매출총이익율,영업이익율,순이익율
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,0.5161,0.0862,0.0648
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,0.1293,-0.0225,-0.0921
2,000050,경방,KOSPI,2022Q1,consolidated_q,0.3623,0.1487,0.0769
3,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,0.2183,0.0983,0.0738
4,000080,하이트진로,KOSPI,2022Q1,consolidated_q,0.4180,0.0796,0.0354
...,...,...,...,...,...,...,...,...
1807,900310,컬러레이,KOSDAQ,2022Q1,consolidated_q,0.6463,0.5224,0.4283
1808,900340,윙입푸드,KOSDAQ,2022Q1,consolidated_q,0.3711,0.1417,0.1222
1809,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,0.7734,0.4290,0.3255
1810,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,0.2002,0.0960,0.0876


In [35]:
Combo2_absolute

,종목코드,종목명,시장,기간,컬럼,PER_score,PBR_score,2Combo
0,124560,태웅로직스,KOSDAQ,2022Q1,consolidated_q,98.8000,88.5000,187.3000
1,084650,랩지노믹스,KOSDAQ,2022Q1,consolidated_q,97.4000,86.5000,183.9000
2,016250,SGC이테크건설,KOSDAQ,2022Q1,consolidated_q,95.6000,94.0000,189.6000
3,041190,우리기술투자,KOSDAQ,2022Q1,consolidated_q,95.1000,68.8000,163.9000
4,036710,심텍홀딩스,KOSDAQ,2022Q1,consolidated_q,94.7000,96.3000,191.0000
...,...,...,...,...,...,...,...,...
344,010780,아이에스동서,KOSPI,2022Q1,consolidated_q,1.6000,77.1000,78.7000
345,141000,비아트론,KOSDAQ,2022Q1,consolidated_q,1.3000,88.0000,89.3000
346,001460,BYC,KOSPI,2022Q1,consolidated_q,0.3000,94.9000,95.2000
347,067310,하나마이크론,KOSDAQ,2022Q1,consolidated_q,0.2000,44.9000,45.1000


In [36]:
Combo2_relative

,종목코드,종목명,시장,기간,컬럼,PER_score,PBR_score,2Combo
0,124560,태웅로직스,KOSDAQ,2022Q1,consolidated_q,99.7768,76.3958,176.1725
1,084650,랩지노믹스,KOSDAQ,2022Q1,consolidated_q,99.5536,72.3675,171.9211
2,016250,SGC이테크건설,KOSDAQ,2022Q1,consolidated_q,98.8839,87.3498,186.2338
3,041190,우리기술투자,KOSDAQ,2022Q1,consolidated_q,98.4375,43.9576,142.3951
4,036710,심텍홀딩스,KOSDAQ,2022Q1,consolidated_q,98.2143,92.4382,190.6524
...,...,...,...,...,...,...,...,...
344,010780,아이에스동서,KOSPI,2022Q1,consolidated_q,0.8929,54.4876,55.3805
345,141000,비아트론,KOSDAQ,2022Q1,consolidated_q,0.6696,75.1943,75.8640
346,001460,BYC,KOSPI,2022Q1,consolidated_q,0.4464,89.0459,89.4924
347,067310,하나마이크론,KOSDAQ,2022Q1,consolidated_q,0.2232,20.1413,20.3646


In [37]:
Combo4

,종목코드,종목명,시장,기간,컬럼,PER_score,PBR_score,PSR_score,PCR_score,4Combo
0,084650,랩지노믹스,KOSDAQ,2022Q1,consolidated_q,99.8424,76.4600,75.3284,94.7781,346.4088
1,036710,심텍홀딩스,KOSDAQ,2022Q1,consolidated_q,99.3696,93.5581,97.0303,96.6928,386.6507
2,104480,티케이케미칼,KOSDAQ,2022Q1,consolidated_q,99.0544,61.8302,61.1079,55.5265,277.5191
3,011200,HMM,KOSPI,2022Q1,consolidated_q,98.5028,51.5352,61.0508,97.3020,308.3908
4,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,98.0299,39.1331,72.4158,99.2167,308.7955
...,...,...,...,...,...,...,...,...,...,...
862,053450,세코닉스,KOSDAQ,2022Q1,consolidated_q,0.7092,53.3414,86.8646,88.6858,229.6010
863,093240,형지엘리트,KOSPI,2022Q1,consolidated_q,0.6304,66.4058,76.9275,74.5866,218.5503
864,013810,스페코,KOSDAQ,2022Q1,consolidated_q,0.3940,46.4178,36.2650,79.0252,162.1021
865,023960,에쓰씨엔지니어링,KOSPI,2022Q1,consolidated_q,0.2364,28.4166,39.0634,8.7032,76.4196


In [38]:
# 모든 팩터 조회
stocks.Factor

,종목코드,종목명,시장,기간,컬럼,PER,EPS,PBR,BPS,SPS,...,EV/Sales,Graham,NCAV,ROA,GP/A,부채비율,차입금비율,매출총이익율,영업이익율,순이익율
0,000020,동화약품,KOSPI,2022Q1,consolidated_q,19.1851,711.4914,0.2676,51003.0442,10972.7129,...,0.6457,-55767848249.9999,516129000000.0001,0.0959,0.0877,26.5437,2.2343,0.5161,0.0862,0.0648
1,000040,KR모터스,KOSPI,2022Q1,consolidated_q,-6.6330,-134.0270,0.4245,2094.2533,1455.8543,...,0.2172,-334921180228.0000,-206722000000.0000,-0.1381,0.0262,243.1920,17.3983,0.1293,-0.0225,-0.0921
2,000050,경방,KOSPI,2022Q1,consolidated_q,14.3040,1115.0720,0.1448,110169.2597,14501.8451,...,2.9755,-2239974334750.0000,-1584064000000.0000,0.3904,0.0282,69.3877,27.3333,0.3623,0.1487,0.0769
3,000070,삼양홀딩스,KOSPI,2022Q1,consolidated_q,3.0925,27841.7159,0.0762,1129448.1457,377025.6686,...,0.4269,-2280545599650.0000,-1174469999999.9998,0.2854,0.0400,82.2613,15.2336,0.2183,0.0983,0.0738
4,000080,하이트진로,KOSPI,2022Q1,consolidated_q,32.9499,1136.5734,0.6064,61759.8315,32103.1524,...,1.1894,-9002475597925.0000,-5062720000000.0000,0.0407,0.0661,228.6561,44.4954,0.4180,0.0796,0.0354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,900310,컬러레이,KOSDAQ,2022Q1,consolidated_q,3.3039,431.3148,0.0874,16307.4815,1007.1111,...,-7.2409,512680000000.0000,628105000000.0000,0.0450,0.0362,10.1416,6.9398,0.6463,0.5224,0.4283
1796,900340,윙입푸드,KOSDAQ,2022Q1,consolidated_q,3.1073,403.8903,0.0983,12761.6688,3306.2886,...,-1.5708,207281021790.0001,297591000000.0001,0.0538,0.0783,22.7849,9.1106,0.3711,0.1417,0.1222
1797,950130,엑세스바이오,KOSDAQ,2022Q1,consolidated_q,1.8501,9729.4471,0.5101,35289.3130,29889.9697,...,-0.3150,-153530228999.9999,826989000000.0001,0.4981,0.3963,65.2944,5.1344,0.7734,0.4290,0.3255
1798,950140,잉글우드랩,KOSDAQ,2022Q1,consolidated_q,11.7770,698.8169,0.5288,15563.0705,7973.9314,...,1.3234,-94301805770.0000,150967000000.0000,0.3555,0.0609,68.4465,26.4604,0.2002,0.0960,0.0876
